In [154]:
# Importar librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import warnings
import time
from selenium.webdriver.common.by import By
import time
import random
from numpy import random as rd
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.common.action_chains import ActionChains

warnings.filterwarnings('ignore')

In [147]:
url = 'https://mmlimp.maps.arcgis.com/apps/webappviewer/index.html?id=ae3b2389664c4e199155e93a6aca9c25#'

## Zonas con vulnerabilidad ante tsunami

In [182]:
# En esta lista se alamacenará todo el contenido de las tablas
list_content = []

In [ ]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get(url)
# Para amximizar la ventana
driver.maximize_window()
time.sleep(rd.randint(2,4,1)[0])
# Cerrar la ventana que aparece al dar click en Aceptar
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Aceptar']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Cerrar el icono de leyenda que aparece
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='close-icon']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Abrir el PLANMET 2040
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='icon-item icon-item-background4 dockable']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Escribir en la casilla de Buscar... (escribir el nombre exacto para que solo salga 1 opción)
search_box = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@class='search-textbox']")))
search_box.send_keys('Zonas con vulnerabilidad ante tsunami')
time.sleep(rd.randint(2,4,1)[0])
# Click en el botón de lupa
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='esri-icon-search']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Click en el botón de agregar
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-dojo-attach-point='addButton']")))
next_button.click()
time.sleep(rd.randint(5,10,1)[0])
# Click para abrir la tabla
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@id='dijit__WidgetBase_0']")))
next_button.click()
time.sleep(rd.randint(5,10,1)[0])
# Click para quitar el filtro predefinido
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@id='dijit_form_ToggleButton_0_label']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])

# Espera hasta que el elemento esté presente
element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='jimu-widget-attributetable-move']"))) 
actions = ActionChains(driver)
# Despalaza hacia arriba 350p
move_up_by = -350
# Realiza la acción de arrastrar y soltar
actions.click_and_hold(element).move_by_offset(0, move_up_by).release().perform()

pagina = bs(driver.page_source,'lxml')

# Para poder identificar la cantidad total de filas que va a descargar (aparece en la parte inferior izquierda de la tabla)
max_fil = int(pagina.find('div',{'class':'dgrid-status self-footer'}).text.split()[0])
#max_fil = 300

scroller = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='dgrid-scroller']")))
driver.execute_script("arguments[0].scrollTop += 1600;", scroller)
time.sleep(rd.randint(3,5,1)[0])
pagina = bs(driver.page_source,'lxml')

# Este for guardará cada registro en la list_content
for i in tqdm(range(1,max_fil+1),desc='Filas: '):
    # Por cada 25 filas se scrollea para que pueda cargar adecuadamente los datos, intenté con 50 y 100 y se pierden :c
    if i%25==0 and (i+25)<max_fil:
        xpath = f"//div[@id='dgrid_0-row-{i+25}']"
        # Espera hasta que el elemento esté presente
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        # Scroll hasta el elemento usando execute_script
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        pagina = bs(driver.page_source,'lxml')
    
    fil = pagina.find('div',{'id':'dgrid_0-row-'+str(i)})
    fila = {
        'OBJECTID': int(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field0 field-objectid field0'}).text),
        'ID_MANZANA': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field1 field-id_manzana field1'}).text,
        'COD_VULNE': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field2 field-cod_vulne field2'}).text,
        'TIPO_VULNE': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field3 field-tipo_vulne field3'}).text,
        'NIV_VULNE': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field4 field-niv_vulne field4'}).text,
        'DESC_VULNE': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field5 field-desc_vulne field5'}).text,
        'NOM_PELI': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field6 field-nom_peli field6'}).text,
        'NOMBDIST': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field7 field-nombdist field7'}).text,
        'NOMBPROV': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field8 field-nombprov field8'}).text,
        'NOMBDEP': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field9 field-nombdep field9'}).text,
        'AREA_HAS': float(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field10 field-area_has field10'}).text.replace('.','').replace(",",".")),
        'SHAPE_AREA': float(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field11 field-Shape__Area field11'}).text.replace('.','').replace(",",".")),
        'SHAPE_LENGHT': float(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field12 field-Shape__Length field12'}).text.replace('.','').replace(",","."))
    }
    list_content.append(fila)

#driver.find_element(By.XPATH, '//*[@id="jimu_dijit_CheckBox_0"]/div[1]').click()
#driver.find_element(by='title',value='Aceptar').click()
html = driver.page_source

In [184]:
# Tabla
df_tsunami= pd.DataFrame(list_content)
df_tsunami

,OBJECTID,ID_MANZANA,COD_VULNE,TIPO_VULNE,NIV_VULNE,DESC_VULNE,NOM_PELI,NOMBDIST,NOMBPROV,NOMBDEP,AREA_HAS,SHAPE_AREA,SHAPE_LENGHT
0,1,15010300200031C,VSITSU-000001,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,ATE,LIMA,LIMA,0.013762,137.62,52.76
1,2,15010302600036B,VSITSU-000002,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,ATE,LIMA,LIMA,0.201682,2016.82,262.12
2,3,150111001000010,VSITSU-000003,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,EL AGUSTINO,LIMA,LIMA,0.266720,2667.20,210.90
3,4,150111001000030,VSITSU-000004,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,EL AGUSTINO,LIMA,LIMA,0.177740,1777.40,170.11
4,5,150111001000040,VSITSU-000005,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,EL AGUSTINO,LIMA,LIMA,0.139178,1391.78,148.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114069,114070,150142031000520,VSITSU-114070,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,VILLA EL SALVADOR,LIMA,LIMA,2.322514,23225.14,611.05
114070,114071,15012801800005A,VSITSU-114071,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,RIMAC,LIMA,LIMA,40.665456,406654.56,3346.39
114071,114072,150131001000130,VSITSU-114072,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,SAN ISIDRO,LIMA,LIMA,0.082167,821.67,118.89
114072,114073,150131002000140,VSITSU-114073,SINTESIS,SIN VULNE,SIN VULNERABILIDAD,TSUNAMI,SAN ISIDRO,LIMA,LIMA,0.069198,691.98,115.77


In [186]:
df_tsunami.to_excel('Zonas vulnerables ante tsunami.xlsx', index=False)

## Cobertura de Agua

In [218]:
# En esta lista se alamacenará todo el contenido de las tablas
list_agua = []

In [222]:
list_agua1 = []

In [223]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get(url)
# Para amximizar la ventana
driver.maximize_window()
time.sleep(rd.randint(2,4,1)[0])
# Cerrar la ventana que aparece al dar click en Aceptar
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Aceptar']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Cerrar el icono de leyenda que aparece
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='close-icon']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Abrir el PLANMET 2040
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='icon-item icon-item-background4 dockable']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Escribir en la casilla de Buscar... (escribir el nombre exacto para que solo salga 1 opción)
search_box = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@class='search-textbox']")))
search_box.send_keys('Cobertura de Agua (PLANMET 2040)')
time.sleep(rd.randint(4,8,1)[0])
# Click en el botón de lupa
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='esri-icon-search']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])

# Click en el botón de agregar
elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@data-dojo-attach-point='addButton']")))
elements[1].click()
time.sleep(rd.randint(5,10,1)[0])
# Click para abrir la tabla
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@id='dijit__WidgetBase_0']")))
next_button.click()
time.sleep(rd.randint(5,10,1)[0])
# Click para quitar el filtro predefinido
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@id='dijit_form_ToggleButton_0_label']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])

# Espera hasta que el elemento esté presente
element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='jimu-widget-attributetable-move']"))) 
actions = ActionChains(driver)
# Despalaza hacia arriba 350p
move_up_by = -350
# Realiza la acción de arrastrar y soltar
actions.click_and_hold(element).move_by_offset(0, move_up_by).release().perform()

pagina = bs(driver.page_source,'lxml')

# Para poder identificar la cantidad total de filas que va a descargar (aparece en la parte inferior izquierda de la tabla)
max_fil = int(pagina.find('div',{'class':'dgrid-status self-footer'}).text.split()[0])
#max_fil = 100

scroller = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='dgrid-scroller']")))
driver.execute_script("arguments[0].scrollTop += 1600;", scroller)
time.sleep(rd.randint(3,5,1)[0])
pagina = bs(driver.page_source,'lxml')

# Este for guardará cada registro en la list_content
for i in tqdm(range(1,max_fil+1),desc='Filas: '):
    # Por cada 25 filas se scrollea para que pueda cargar adecuadamente los datos, intenté con 50 y 100 y se pierden :c
    if i%25==0 and (i+25)<=max_fil:
        xpath = f"//div[@id='dgrid_0-row-{i+25}']"
        # Espera hasta que el elemento esté presente
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        # Scroll hasta el elemento usando execute_script
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        pagina = bs(driver.page_source,'lxml')

    if i > 97049:   
        fil = pagina.find('div',{'id':'dgrid_0-row-'+str(i)})
        fila = {
            'OBJECTID': int(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field0 field-objectid field0'}).text),
            'CODLOTE': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field1 field-codlote field1'}).text,
            'IDLOTE': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field2 field-idlote field2'}).text,
            'ID_MANZANA': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field3 field-id_manzana field3'}).text,
            'IDMAZNAR': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field4 field-idmznar field4'}).text,
            'COB_AP17': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field5 field-cob_ap17 field5'}).text,
            'AREA_M2': float(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field6 field-area_m2 field6'}).text.replace('.','').replace(",",".")),
            'NOMBDIST': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field7 field-nombdist field7'}).text,
            'NOMBPROV': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field8 field-nombprov field8'}).text,
            'NOMBDEP': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field9 field-nombdep field9'}).text,
            'ID_SECT': fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field10 field-id_sect field10'}).text,
            'NOM_UA': fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field11 field-nom_ua field11'}).text,
            'SHAPE_AREA': float(fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field12 field-Shape__Area field12'}).text.replace('.','').replace(",",".")),
            'SHAPE_LENGHT': float(fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field13 field-Shape__Length field13'}).text.replace('.','').replace(",","."))
        }
        list_agua1.append(fila)

#driver.find_element(By.XPATH, '//*[@id="jimu_dijit_CheckBox_0"]/div[1]').click()
#driver.find_element(by='title',value='Aceptar').click()
html = driver.page_source

Filas: 100%|███████████████████████████████████████████████████████████████████| 112261/112261 [55:12<00:00, 33.89it/s]


In [214]:
fil = pagina.find('div',{'id':'dgrid_0-row-1'})
float(fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field13 field-Shape__Length field13'}).text.replace('.','').replace(",","."))

231.66

In [220]:
# Tabla de agua
df_agua= pd.DataFrame(list_agua)
df_agua

,OBJECTID,CODLOTE,IDLOTE,ID_MANZANA,IDMAZNAR,COB_AP17,AREA_M2,NOMBDIST,NOMBPROV,NOMBDEP,ID_SECT,NOM_UA,SHAPE_AREA,SHAPE_LENGHT
0,1,,,150101001000010,,CON COBERTURA,2509.72,LIMA CERCADO,LIMA,LIMA,,,2509.72,231.66
1,2,,,150101001000020,,CON COBERTURA,2015.30,LIMA CERCADO,LIMA,LIMA,,,2015.30,201.90
2,3,,,150101001000030,,CON COBERTURA,9710.06,LIMA CERCADO,LIMA,LIMA,,,9710.06,793.55
3,4,,,150101001000040,,CON COBERTURA,5408.58,LIMA CERCADO,LIMA,LIMA,,,5408.58,303.74
4,5,,,150101001000050,,CON COBERTURA,5066.66,LIMA CERCADO,LIMA,LIMA,,,5066.67,301.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97044,97045,,,150142031000420,,CON COBERTURA,3664.18,VILLA EL SALVADOR,LIMA,LIMA,,,3664.18,261.92
97045,97046,,,15014203100043A,,NO DATA,4036.86,VILLA EL SALVADOR,LIMA,LIMA,,,4036.86,269.51
97046,97047,,,15014203100043B,,NO DATA,3823.57,VILLA EL SALVADOR,LIMA,LIMA,,,3823.57,264.80
97047,97048,,,150142031000440,,CON COBERTURA,3462.45,VILLA EL SALVADOR,LIMA,LIMA,,,3462.45,257.54


In [225]:
df_agua2= pd.DataFrame(list_agua1)
df_agua2

,OBJECTID,CODLOTE,IDLOTE,ID_MANZANA,IDMAZNAR,COB_AP17,AREA_M2,NOMBDIST,NOMBPROV,NOMBDEP,ID_SECT,NOM_UA,SHAPE_AREA,SHAPE_LENGHT
0,97050,,,150142031000460,,CON COBERTURA,3641.27,VILLA EL SALVADOR,LIMA,LIMA,,,3641.27,259.64
1,97051,,,150142031000470,,CON COBERTURA,3697.98,VILLA EL SALVADOR,LIMA,LIMA,,,3697.98,258.83
2,97052,,,150142031000480,,CON COBERTURA,3588.86,VILLA EL SALVADOR,LIMA,LIMA,,,3588.86,257.13
3,97053,,,150142031000490,,CON COBERTURA,3491.84,VILLA EL SALVADOR,LIMA,LIMA,,,3491.85,254.82
4,97054,,,150142031000500,,CON COBERTURA,3738.62,VILLA EL SALVADOR,LIMA,LIMA,,,3738.62,261.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15207,112257,,,,MZ-0000054,NO DATA,3689.63,ANCON,LIMA,LIMA,,,3689.63,301.55
15208,112258,,,,MZ-0000053,NO DATA,3539.50,ANCON,LIMA,LIMA,,,3539.50,292.96
15209,112259,,,,MZ-0000071,NO DATA,5110.19,ANCON,LIMA,LIMA,,,5110.19,365.84
15210,112260,,,,MZ-0000058,NO DATA,4676.34,ANCON,LIMA,LIMA,,,4676.34,363.98


In [230]:
df_aguafinal = pd.concat([df_agua, df_agua2], axis=0)
df_aguafinal.reset_index(drop=True, inplace=True)

In [231]:
df_aguafinal

,OBJECTID,CODLOTE,IDLOTE,ID_MANZANA,IDMAZNAR,COB_AP17,AREA_M2,NOMBDIST,NOMBPROV,NOMBDEP,ID_SECT,NOM_UA,SHAPE_AREA,SHAPE_LENGHT
0,1,,,150101001000010,,CON COBERTURA,2509.72,LIMA CERCADO,LIMA,LIMA,,,2509.72,231.66
1,2,,,150101001000020,,CON COBERTURA,2015.30,LIMA CERCADO,LIMA,LIMA,,,2015.30,201.90
2,3,,,150101001000030,,CON COBERTURA,9710.06,LIMA CERCADO,LIMA,LIMA,,,9710.06,793.55
3,4,,,150101001000040,,CON COBERTURA,5408.58,LIMA CERCADO,LIMA,LIMA,,,5408.58,303.74
4,5,,,150101001000050,,CON COBERTURA,5066.66,LIMA CERCADO,LIMA,LIMA,,,5066.67,301.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112256,112257,,,,MZ-0000054,NO DATA,3689.63,ANCON,LIMA,LIMA,,,3689.63,301.55
112257,112258,,,,MZ-0000053,NO DATA,3539.50,ANCON,LIMA,LIMA,,,3539.50,292.96
112258,112259,,,,MZ-0000071,NO DATA,5110.19,ANCON,LIMA,LIMA,,,5110.19,365.84
112259,112260,,,,MZ-0000058,NO DATA,4676.34,ANCON,LIMA,LIMA,,,4676.34,363.98


In [232]:
df_aguafinal.to_excel('Cobertura de Agua.xlsx', index=False)

## Áreas verdes

In [235]:
list_verde = []

In [ ]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get(url)
# Para amximizar la ventana
driver.maximize_window()
time.sleep(rd.randint(2,4,1)[0])
# Cerrar la ventana que aparece al dar click en Aceptar
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Aceptar']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Cerrar el icono de leyenda que aparece
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='close-icon']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Abrir el PLANMET 2040
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='icon-item icon-item-background4 dockable']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])
# Escribir en la casilla de Buscar... (escribir el nombre exacto para que solo salga 1 opción)
search_box = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@class='search-textbox']")))
search_box.send_keys('áreas verdes')
time.sleep(rd.randint(4,8,1)[0])
# Click en el botón de lupa
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='esri-icon-search']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])

# Click en el botón de agregar
elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@data-dojo-attach-point='addButton']")))
elements[1].click()
time.sleep(rd.randint(5,10,1)[0])
# Click para abrir la tabla
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@id='dijit__WidgetBase_0']")))
next_button.click()
time.sleep(rd.randint(5,10,1)[0])
# Click para quitar el filtro predefinido
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@id='dijit_form_ToggleButton_0_label']")))
next_button.click()
time.sleep(rd.randint(2,4,1)[0])

# Espera hasta que el elemento esté presente
element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='jimu-widget-attributetable-move']"))) 
actions = ActionChains(driver)
# Despalaza hacia arriba 350p
move_up_by = -350
# Realiza la acción de arrastrar y soltar
actions.click_and_hold(element).move_by_offset(0, move_up_by).release().perform()

pagina = bs(driver.page_source,'lxml')

# Para poder identificar la cantidad total de filas que va a descargar (aparece en la parte inferior izquierda de la tabla)
max_fil = int(pagina.find('div',{'class':'dgrid-status self-footer'}).text.split()[0])
#max_fil = 100

scroller = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='dgrid-scroller']")))
driver.execute_script("arguments[0].scrollTop += 1600;", scroller)
time.sleep(rd.randint(3,5,1)[0])
pagina = bs(driver.page_source,'lxml')

# Este for guardará cada registro en la list_content
for i in tqdm(range(1,max_fil+1),desc='Filas: '):
    # Por cada 25 filas se scrollea para que pueda cargar adecuadamente los datos, intenté con 50 y 100 y se pierden :c
    if i%25==0 and (i+25)<=max_fil:
        xpath = f"//div[@id='dgrid_0-row-{i+25}']"
        # Espera hasta que el elemento esté presente
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        # Scroll hasta el elemento usando execute_script
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        pagina = bs(driver.page_source,'lxml')
    
    fil = pagina.find('div',{'id':'dgrid_0-row-'+str(i)})
    fila = {
        'OBJECTID': int(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field0 field-objectid field0'}).text),
        'COD_AV': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field1 field-cod_av field1'}).text,
        'NOM_AV': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field2 field-nom_av field2'}).text,
        'AREA_HAS': float(fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field3 field-area_has field3'}).text.replace('.','').replace(",",".")),
        'NOMBDIST': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field4 field-nombdist field4'}).text,
        'NOMBPROV': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field5 field-nombprov field5'}).text,
        'NOMBDEP': fil.find('td',{'class':'dgrid-cell dgrid-cell-padding dgrid-column-field6 field-nombdep field6'}).text,
        'NOM_EQ': fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field7 field-nom_eq field7'}).text,
        'SHAPE_AREA': float(fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field8 field-Shape__Area field8'}).text.replace('.','').replace(",",".")),
        'SHAPE_LENGHT': float(fil.find('td',{'dgrid-cell dgrid-cell-padding dgrid-column-field9 field-Shape__Length field9'}).text.replace('.','').replace(",","."))
    }
    list_verde.append(fila)

#driver.find_element(By.XPATH, '//*[@id="jimu_dijit_CheckBox_0"]/div[1]').click()
#driver.find_element(by='title',value='Aceptar').click()
html = driver.page_source

In [237]:
df_area_Verde= pd.DataFrame(list_verde)
df_area_Verde

,OBJECTID,COD_AV,NOM_AV,AREA_HAS,NOMBDIST,NOMBPROV,NOMBDEP,NOM_EQ,SHAPE_AREA,SHAPE_LENGHT
0,1,AV-0001,AREAS VERDES,0.252186,PUENTE PIEDRA,LIMA,LIMA,,2521.86,397.11
1,2,AV-0002,AREAS VERDES,0.541773,PUENTE PIEDRA,LIMA,LIMA,PLAZA VIRGEN DE LA MEDALLA MILAGROSA,5417.73,329.34
2,3,AV-0003,AREAS VERDES,0.472750,PUENTE PIEDRA,LIMA,LIMA,PLAZA JERUSALÉN,4727.50,275.62
3,4,AV-0004,AREAS VERDES,0.330861,PUENTE PIEDRA,LIMA,LIMA,"PARQUE DE ""CALICHE""",3308.61,231.40
4,5,AV-0005,AREAS VERDES,0.159632,PUENTE PIEDRA,LIMA,LIMA,PARQUE MIRAFLORES,1596.32,160.91
...,...,...,...,...,...,...,...,...,...,...
8320,8321,AV-8321,AREAS VERDES,0.350461,CHORRILLOS,LIMA,LIMA,,3504.61,1561.05
8321,8322,AV-8322,AREAS VERDES,0.069647,SANTIAGO DE SURCO,LIMA,LIMA,,696.47,189.35
8322,8323,AV-8323,AREAS VERDES,8.742018,LOS OLIVOS,LIMA,LIMA,,87420.18,1598.07
8323,8324,AV-8324,AREAS VERDES,0.106923,COMAS,LIMA,LIMA,,1069.24,161.98


In [238]:
df_area_Verde.to_excel('Areas verdes.xlsx', index=False)